In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**TASKS**

1. Always normalize data. - use Z scaling. 
2. eliminate rows with greater than 5 sig deviation
3. compare the statistics for train, val and test
4. One hot encoding for coupling type
5. Check if Electronegativity is a necessary measure - Khan Academy
5. Check if size and type of atom influence the value of coupling constant - Khan Academy and others
6. Test to see if separating the prediction coupling types is a good idea
7. Investigate corrolations
8. HOW DO NEARBY ATOMS AFFECT THE DISTRIBUTION? ELECTRONEGATIVITY?

**NOTES:**

1. No corrolation between mulliken charge and scalar coupling constant
2. No corrolation between mulliken charge and coupling type
3. May need to isolate certain types of coupling based on distribution of scalar coupling. Refer violin plot of distribution of scalar coupling constant


In [ ]:
#libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def normalise (x):
    temp = pd.Series()
    temp = (x - x.mean())/x.std()
    return temp, x.mean(), x.std()
    

In [ ]:
X.columns

In [ ]:
X.atom_x.unique(), X.atom_y.unique(), X.type.unique()

In [ ]:
def nega (x):
    nega_dict = {'C':2.55, 'H':2.2, 'N':3.04, 'O':3.44, 'F':3.98}
    return [nega_dict[var] for var in x]

In [ ]:
def radii (x):
    radi_dict = {'C':2.55, 'H':2.2, 'N':3.04, 'O':3.44, 'F':3.98}

In [ ]:
structure.atom.unique()

In [ ]:
# nega_dict = {'C':2.55, 'H':2.2, 'N':3.04, 'O':3.44, 'F':3.98}
temp = ['C','C','H','N','O','F']
nega(temp)

In [ ]:
structure.head()

In [ ]:
""" LOG MAE score calculator
    taken from the public kernel - https://www.kaggle.com/marcelotamashiro/lgb-public-kernels-plus-more-features @marcelotomashiro
"""

def comp_score (y_true, y_pred, jtype):
    df = pd.DataFrame()
    df['y_true'] , df['y_pred'], df['jtype'] = y_true , y_pred, jtype
    score = 0 
    for t in jtype.unique():
        score_jtype = np.log(mean_absolute_error(df[df.jtype==t]['y_true'],df[df.jtype==t]['y_pred']))
        score += score_jtype
        print(f'{t} : {score_jtype}')
    score /= len(jtype.unique())
    return score

In [ ]:
train = pd.read_csv("../input/train.csv")
structure = pd.read_csv("../input/structures.csv")

X = train[train.columns[1:-1]]
y = train[train.columns[-1]]

X = pd.merge(X,structure, left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'], how='left')
X = pd.merge(X,structure, left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'], how='left')
X = X.drop(['atom_index_x','atom_index_y'], axis =1)

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.3)

In [ ]:
### import additional stuff
mulliken_charges = pd.read_csv("../input/mulliken_charges.csv")
mulliken_charges.columns

In [ ]:
test = pd.read_csv("../input/test.csv")
X_test, y_test = test[test.columns[1:-1]],test[test.columns[-1]]

In [ ]:
train.type.value_counts()

In [ ]:
X_m = pd.DataFrame()
X_m = pd.merge(X, mulliken_charges, left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'], how='left')
X_m = pd.merge(X_m, mulliken_charges, left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'], how='left')
X_m = X_m.drop(['atom_index_x','atom_index_y'], axis =1)

X_m.head()


In [ ]:
structure.atom.unique()
# X.atom_y.unique()


In [ ]:
X_m.type.unique()

In [ ]:
X_m_temp = X_m.loc[X_m.type=='2JHN']
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_m_temp.x_x, X_m_temp.y_x, y[X_m_temp.index])

In [ ]:
X_m.mulliken_charge_x.max() - X_m.mulliken_charge_x.min()

In [ ]:
plt.pyplot.scatter(np.abs([X_m.mulliken_charge_y[::500] - X_m.mulliken_charge_x[::500]]), y[::500])

In [ ]:
sns.violinplot(X_m.type[::500], np.abs(X_m.mulliken_charge_x[::500] - X_m.mulliken_charge_y[::500]))

In [ ]:
structure.head()

In [ ]:
# bond length calculation
def bondlength (X, X_structures):
# calculate cartesian distance    
    X  
    

In [ ]:
[x for x in range(5)]

In [ ]:
lista = [x for x in range(len(train))]

In [ ]:
temp = train.assign(atom_0 = lista)
temp.head()

In [ ]:
structure.head()

In [ ]:
structure.x.loc[(structure.molecule_name == 'dsgdb9nsd_000001') & (structure.atom == 'C')]

In [ ]:
var = 'x'
structure[var].loc[(structure.molecule_name == 'dsgdb9nsd_000001') & (structure.atom == 'C')]

In [ ]:
def newfeature(X, struct, target, filter_1, filter_2, filter_3):
    temp = list()
    for index, molecule in X.iterrows():
        temp.append(struct[target].loc((struct[filter_1] == molecule[filter_1]) & (struct[filter_3] == molecule[filter_2])))
    
    return temp
        

In [ ]:
# atom_0_x, atom_0_y, atom_0 = some function

train_compd = train.assign(atom_0_x = newfeature(train,structure,'x','molecule_name','atom_index_0','atom_index'))

In [ ]:
# IGNORE
compendium = pd.DataFrame()
for molecule in train.molecule_name:
    train.loc[train.molecule_name==molecule].insert({'bondlength':bondlength(train.loc[train.molecule_name==molecule], 
                                                                            structures.loc[structures.molecule_name==molecule])})

In [ ]:
temp,scc_mean, scc_std = normalise(train.scalar_coupling_constant)

In [ ]:
import seaborn as sns

In [ ]:
# log transformation
log_temp = np.log(temp - temp.min())

In [ ]:
temp.loc[temp>1.5].count()/temp.count()

**RESIDUAL CODE AFTER THIS POINT**

In [ ]:
train.loc[train.molecule_name=='dsgdb9nsd_000001']

In [ ]:
train.loc[train.type=='2JHH'].index.max()

In [ ]:
np.sqrt(5)

In [ ]:
X.columns

In [ ]:
X.x_x.head()

In [ ]:
def bond_length(var):
    return np.sqrt((var.x_x - var.x_y)**2 + (var.y_x - var.y_y)**2 + (var.z_x - var.z_y)**2)
    

In [ ]:
X = X.assign(bond_length = bond_length(X))

In [ ]:
train_sub = train.loc[(train.type!='1JHC') & (train.type!='1JHN')]

In [ ]:
y_sub_norm, sub_mean, sub_std = normalise(train_sub.scalar_coupling_constant)

In [ ]:
y_sub_norm.describe()

In [ ]:
y_sub_norm_red = y_sub_norm.loc[(y_sub_norm >= -4) & (y_sub_norm <= 4)]

In [ ]:
y_sub_norm_red.shape[0] - y_sub_norm.shape[0]

In [ ]:
train_sub.type.value_counts()

In [ ]:
train_sub.loc[y_sub_norm_red.index].type.value_counts()

In [ ]:
sns.violinplot(train.type,train.scalar_coupling_constant)

In [ ]:
X.loc

In [ ]:
X.loc[X.type=='3JHN']

In [ ]:
var = 7
sns.scatterplot(X.bond_length.loc[X.type == X.type.unique()[var]], y.iloc[X.loc[X.type == X.type.unique()[var]].index] )

In [ ]:
sns.violinplot(X.type, X.bond_length)

In [ ]:
"""Train set minus three types - 1JHC, 2JHH, 1JHN

"""
train_minus_three = train.loc[ (train.type!= '1JHC') & (train.type!= '2JHH') & (train.type!= '1JHN')]
y_minus, minus_mean, minus_std = normalise(train_minus_three.scalar_coupling_constant)
train_minus_red = train_minus_three.loc[y_minus.loc[(y_minus >= -5) & (y_minus<= 5)].index]

train_plus_three = train.loc[ (train.type == '1JHC') | (train.type == '2JHH') | (train.type == '1JHN')]
y_plus, plus_mean, plus_std = normalise(train_plus_three.scalar_coupling_constant)
train_plus_red = train_plus_three.loc[y_plus.loc[(y_plus >= -5) & (y_plus <= 5)].index]

In [ ]:
train_plus_red.type.value_counts()

In [ ]:
train_plus_three.type.value_counts()

In [ ]:
sns.distplot(y_plus, kde=True)

In [ ]:
sns.distplot(y_minus.loc[(y_minus >= -5) & (y_minus<= 5)], kde=True)

In [ ]:
sns.distplot(y_minus, kde=True)

In [ ]:
sns.distplot(y_sub_norm_red, kde= True)

In [ ]:
sns.distplot(y_sub_norm, kde=True)